In [1]:
from mpmath.libmp.libintmath import _1_100
import sympy as sp
from sympy import N, pprint, print_latex
from sympy.parsing.mathematica import parse_mathematica

In [2]:
def full_expression_sorted(L, K):
    if L <= 0 or K <= 0:
        raise ValueError("Matrix dimensions must be positive integers.")
    A = sp.Matrix([[sp.symbols(f'a{i+1}{j+1}') for j in range(L)] for i in range(L)])
    E = sp.Matrix([[sp.symbols(f'e{i+j}') for j in range(K)] for i in range(L)])
    B = sp.Matrix([[sp.symbols(f'b{i+1}{j+1}') for j in range(K)] for i in range(K)])
    Y = sp.Matrix(L, K, lambda i, j: 0)  # Initialize result matrix
    for i in range(L):
        for j in range(K):
            AE_term = sum(A[i, k] * E[k, j] for k in range(L))
            EB_term = sum(E[i, k] * B[k, j] for k in range(K))
            AEB_term = sum(sum(A[i, k] * E[k, l] * B[l, j] for k in range(L)) for l in range(K))
            #Combine terms BEFORE applying collect to handle parentheses correctly
            combined_term = AE_term + EB_term - AEB_term
            # Correct range for k and apply collect
            Y[i, j] = sp.collect(combined_term, [sp.Symbol(f"e{k}") for k in range(L + K - 1)])

    return Y

In [3]:
def full_expression_diagonal_averaging(L, K):
    if L <= 0 or K <= 0:
        raise ValueError("Matrix dimensions must be positive integers.")

    A = sp.Matrix([[sp.symbols(f'a{i+1}{j+1}') for j in range(L)] for i in range(L)])
    E = sp.Matrix([[sp.symbols(f'e{i+j}') for j in range(K)] for i in range(L)])
    B = sp.Matrix([[sp.symbols(f'b{i+1}{j+1}') for j in range(K)] for i in range(K)])
    
    Y = A*E + E*B - A*E*B
    if Y is None:  # Handle potential errors from matrix_products
        return None

    N = L + K - 1
    g_k = []
    ek_symbols = [sp.Symbol(f'e{k}') for k in range(N)]

    for k in range(N):
        sum_term = 0
        if k < L - 1:
            for m in range(1, k + 2):
                sum_term += Y[m - 1, k - m + 1]
            gk = sp.Rational(1, (k + 1)) * sum_term
        elif k < K:
            for m in range(1, L + 1):
                sum_term += Y[m - 1, k - m + 1]
            gk = sp.Rational(1, L) * sum_term
        else:
            for m in range(k - K + 2, N - K + 2):
                sum_term += Y[m - 1, k - m + 1]
            gk = sp.Rational(1, (N - k)) * sum_term

        expanded_gk = sp.expand(gk)
        g_k.append(sp.collect(expanded_gk, ek_symbols))
        
    return g_k


In [4]:
def make_C_matrix(g_k):
    N = len(g_k)
    ek_symbols = [sp.Symbol(f'e{k}') for k in range(N)]
    C_matrix = sp.Matrix(N, N, lambda i, j: 0)
    for i in range(N):
        coeff_dict = sp.Poly(g_k[i],*ek_symbols).as_poly().coeffs()
        for j in range(len(coeff_dict)):
            C_matrix[i,j] = coeff_dict[j]
    return C_matrix

In [5]:
L = 5
K = 5
Y = full_expression_sorted(L, K)
print("\nAE + EB - AEB (sorted by e_k):")
Y


AE + EB - AEB (sorted by e_k):


Matrix([
[  -a15*b51*e8 + e0*(-a11*b11 + a11 + b11) + e1*(-a11*b21 - a12*b11 + a12 + b21) + e2*(-a11*b31 - a12*b21 - a13*b11 + a13 + b31) + e3*(-a11*b41 - a12*b31 - a13*b21 - a14*b11 + a14 + b41) + e4*(-a11*b51 - a12*b41 - a13*b31 - a14*b21 - a15*b11 + a15 + b51) + e5*(-a12*b51 - a13*b41 - a14*b31 - a15*b21) + e6*(-a13*b51 - a14*b41 - a15*b31) + e7*(-a14*b51 - a15*b41), -a15*b52*e8 + e0*(-a11*b12 + b12) + e1*(-a11*b22 + a11 - a12*b12 + b22) + e2*(-a11*b32 - a12*b22 + a12 - a13*b12 + b32) + e3*(-a11*b42 - a12*b32 - a13*b22 + a13 - a14*b12 + b42) + e4*(-a11*b52 - a12*b42 - a13*b32 - a14*b22 + a14 - a15*b12 + b52) + e5*(-a12*b52 - a13*b42 - a14*b32 - a15*b22 + a15) + e6*(-a13*b52 - a14*b42 - a15*b32) + e7*(-a14*b52 - a15*b42), -a15*b53*e8 + e0*(-a11*b13 + b13) + e1*(-a11*b23 - a12*b13 + b23) + e2*(-a11*b33 + a11 - a12*b23 - a13*b13 + b33) + e3*(-a11*b43 - a12*b33 + a12 - a13*b23 - a14*b13 + b43) + e4*(-a11*b53 - a12*b43 - a13*b33 + a13 - a14*b23 - a15*b13 + b53) + e5*(-a12*b53 - a13*b43 -

In [6]:
L = 5
K = 5
g_k = full_expression_diagonal_averaging(L, K)
print("\nAE + EB - AEB (sorted by e_k):")
for i in range(len(g_k)):
    print(g_k[i])



AE + EB - AEB (sorted by e_k):
-a15*b51*e8 + e0*(-a11*b11 + a11 + b11) + e1*(-a11*b21 - a12*b11 + a12 + b21) + e2*(-a11*b31 - a12*b21 - a13*b11 + a13 + b31) + e3*(-a11*b41 - a12*b31 - a13*b21 - a14*b11 + a14 + b41) + e4*(-a11*b51 - a12*b41 - a13*b31 - a14*b21 - a15*b11 + a15 + b51) + e5*(-a12*b51 - a13*b41 - a14*b31 - a15*b21) + e6*(-a13*b51 - a14*b41 - a15*b31) + e7*(-a14*b51 - a15*b41)
e0*(-a11*b12/2 - a21*b11/2 + a21/2 + b12/2) + e1*(-a11*b22/2 + a11/2 - a12*b12/2 - a21*b21/2 - a22*b11/2 + a22/2 + b11/2 + b22/2) + e2*(-a11*b32/2 - a12*b22/2 + a12/2 - a13*b12/2 - a21*b31/2 - a22*b21/2 - a23*b11/2 + a23/2 + b21/2 + b32/2) + e3*(-a11*b42/2 - a12*b32/2 - a13*b22/2 + a13/2 - a14*b12/2 - a21*b41/2 - a22*b31/2 - a23*b21/2 - a24*b11/2 + a24/2 + b31/2 + b42/2) + e4*(-a11*b52/2 - a12*b42/2 - a13*b32/2 - a14*b22/2 + a14/2 - a15*b12/2 - a21*b51/2 - a22*b41/2 - a23*b31/2 - a24*b21/2 - a25*b11/2 + a25/2 + b41/2 + b52/2) + e5*(-a12*b52/2 - a13*b42/2 - a14*b32/2 - a15*b22/2 + a15/2 - a22*b51/2 - a

In [7]:
C = make_C_matrix(g_k)
C

Matrix([
[                                                      -a11*b11 + a11 + b11,                                                                                                                         -a11*b21 - a12*b11 + a12 + b21,                                                                                                                                                                                           -a11*b31 - a12*b21 - a13*b11 + a13 + b31,                                                                                                                                                                                                                                                             -a11*b41 - a12*b31 - a13*b21 - a14*b11 + a14 + b41,                                                                                                                                                                                                                                       

In [8]:
L=K=5
A = sp.Matrix([[sp.symbols(f'a{i+1}{j+1}') for j in range(L)] for i in range(L)])
E = sp.Matrix([[sp.symbols(f'e{i+j}') for j in range(K)] for i in range(L)])
B = sp.Matrix([[sp.symbols(f'b{i+1}{j+1}') for j in range(K)] for i in range(K)])

A, B, E

(Matrix([
 [a11, a12, a13, a14, a15],
 [a21, a22, a23, a24, a25],
 [a31, a32, a33, a34, a35],
 [a41, a42, a43, a44, a45],
 [a51, a52, a53, a54, a55]]),
 Matrix([
 [b11, b12, b13, b14, b15],
 [b21, b22, b23, b24, b25],
 [b31, b32, b33, b34, b35],
 [b41, b42, b43, b44, b45],
 [b51, b52, b53, b54, b55]]),
 Matrix([
 [e0, e1, e2, e3, e4],
 [e1, e2, e3, e4, e5],
 [e2, e3, e4, e5, e6],
 [e3, e4, e5, e6, e7],
 [e4, e5, e6, e7, e8]]))

In [10]:
AE=A*E
EB=E*B
AEB=A*E*B
print_latex(AE, mat_str="pmatrix")
EB
AEB

\left[\begin{pmatrix}a_{11} e_{0} + a_{12} e_{1} + a_{13} e_{2} + a_{14} e_{3} + a_{15} e_{4} & a_{11} e_{1} + a_{12} e_{2} + a_{13} e_{3} + a_{14} e_{4} + a_{15} e_{5} & a_{11} e_{2} + a_{12} e_{3} + a_{13} e_{4} + a_{14} e_{5} + a_{15} e_{6} & a_{11} e_{3} + a_{12} e_{4} + a_{13} e_{5} + a_{14} e_{6} + a_{15} e_{7} & a_{11} e_{4} + a_{12} e_{5} + a_{13} e_{6} + a_{14} e_{7} + a_{15} e_{8}\\a_{21} e_{0} + a_{22} e_{1} + a_{23} e_{2} + a_{24} e_{3} + a_{25} e_{4} & a_{21} e_{1} + a_{22} e_{2} + a_{23} e_{3} + a_{24} e_{4} + a_{25} e_{5} & a_{21} e_{2} + a_{22} e_{3} + a_{23} e_{4} + a_{24} e_{5} + a_{25} e_{6} & a_{21} e_{3} + a_{22} e_{4} + a_{23} e_{5} + a_{24} e_{6} + a_{25} e_{7} & a_{21} e_{4} + a_{22} e_{5} + a_{23} e_{6} + a_{24} e_{7} + a_{25} e_{8}\\a_{31} e_{0} + a_{32} e_{1} + a_{33} e_{2} + a_{34} e_{3} + a_{35} e_{4} & a_{31} e_{1} + a_{32} e_{2} + a_{33} e_{3} + a_{34} e_{4} + a_{35} e_{5} & a_{31} e_{2} + a_{32} e_{3} + a_{33} e_{4} + a_{34} e_{5} + a_{35} e_{6} & a_{31}

Matrix([
[b11*(a11*e0 + a12*e1 + a13*e2 + a14*e3 + a15*e4) + b21*(a11*e1 + a12*e2 + a13*e3 + a14*e4 + a15*e5) + b31*(a11*e2 + a12*e3 + a13*e4 + a14*e5 + a15*e6) + b41*(a11*e3 + a12*e4 + a13*e5 + a14*e6 + a15*e7) + b51*(a11*e4 + a12*e5 + a13*e6 + a14*e7 + a15*e8), b12*(a11*e0 + a12*e1 + a13*e2 + a14*e3 + a15*e4) + b22*(a11*e1 + a12*e2 + a13*e3 + a14*e4 + a15*e5) + b32*(a11*e2 + a12*e3 + a13*e4 + a14*e5 + a15*e6) + b42*(a11*e3 + a12*e4 + a13*e5 + a14*e6 + a15*e7) + b52*(a11*e4 + a12*e5 + a13*e6 + a14*e7 + a15*e8), b13*(a11*e0 + a12*e1 + a13*e2 + a14*e3 + a15*e4) + b23*(a11*e1 + a12*e2 + a13*e3 + a14*e4 + a15*e5) + b33*(a11*e2 + a12*e3 + a13*e4 + a14*e5 + a15*e6) + b43*(a11*e3 + a12*e4 + a13*e5 + a14*e6 + a15*e7) + b53*(a11*e4 + a12*e5 + a13*e6 + a14*e7 + a15*e8), b14*(a11*e0 + a12*e1 + a13*e2 + a14*e3 + a15*e4) + b24*(a11*e1 + a12*e2 + a13*e3 + a14*e4 + a15*e5) + b34*(a11*e2 + a12*e3 + a13*e4 + a14*e5 + a15*e6) + b44*(a11*e3 + a12*e4 + a13*e5 + a14*e6 + a15*e7) + b54*(a11*e4 + a12*e5 + a

In [12]:
def diagonal_averaging(mat):
    L,K = mat.shape
    N = L + K - 1
    g_k = []
    ek_symbols = [sp.Symbol(f'e{k}') for k in range(N)]
    for k in range(N):
        sum_term = 0
        if k < L - 1:
            for m in range(1, k + 2):
                sum_term += mat[m - 1, k - m + 1]
            gk = sp.Rational(1, (k + 1)) * sum_term
        elif k < K:
            for m in range(1, L + 1):
                sum_term += mat[m - 1, k - m + 1]
            gk = sp.Rational(1, L) * sum_term
        else:
            for m in range(k - K + 2, N - K + 2):
                sum_term += mat[m - 1, k - m + 1]
            gk = sp.Rational(1, (N - k)) * sum_term

        expanded_gk = sp.expand(gk)
        g_k.append(sp.collect(expanded_gk, ek_symbols))
        
    return g_k

In [16]:
ga_k = diagonal_averaging(AE)
print_latex(ga_k)

\left[ a_{11} e_{0} + a_{12} e_{1} + a_{13} e_{2} + a_{14} e_{3} + a_{15} e_{4}, \  \frac{a_{15} e_{5}}{2} + \frac{a_{21} e_{0}}{2} + e_{1} \left(\frac{a_{11}}{2} + \frac{a_{22}}{2}\right) + e_{2} \left(\frac{a_{12}}{2} + \frac{a_{23}}{2}\right) + e_{3} \left(\frac{a_{13}}{2} + \frac{a_{24}}{2}\right) + e_{4} \left(\frac{a_{14}}{2} + \frac{a_{25}}{2}\right), \  \frac{a_{15} e_{6}}{3} + \frac{a_{31} e_{0}}{3} + e_{1} \left(\frac{a_{21}}{3} + \frac{a_{32}}{3}\right) + e_{2} \left(\frac{a_{11}}{3} + \frac{a_{22}}{3} + \frac{a_{33}}{3}\right) + e_{3} \left(\frac{a_{12}}{3} + \frac{a_{23}}{3} + \frac{a_{34}}{3}\right) + e_{4} \left(\frac{a_{13}}{3} + \frac{a_{24}}{3} + \frac{a_{35}}{3}\right) + e_{5} \left(\frac{a_{14}}{3} + \frac{a_{25}}{3}\right), \  \frac{a_{15} e_{7}}{4} + \frac{a_{41} e_{0}}{4} + e_{1} \left(\frac{a_{31}}{4} + \frac{a_{42}}{4}\right) + e_{2} \left(\frac{a_{21}}{4} + \frac{a_{32}}{4} + \frac{a_{43}}{4}\right) + e_{3} \left(\frac{a_{11}}{4} + \frac{a_{22}}{4} + \frac{a_{

In [17]:
gb_k = diagonal_averaging(EB)
print_latex(gb_k)

\left[ b_{11} e_{0} + b_{21} e_{1} + b_{31} e_{2} + b_{41} e_{3} + b_{51} e_{4}, \  \frac{b_{12} e_{0}}{2} + \frac{b_{51} e_{5}}{2} + e_{1} \left(\frac{b_{11}}{2} + \frac{b_{22}}{2}\right) + e_{2} \left(\frac{b_{21}}{2} + \frac{b_{32}}{2}\right) + e_{3} \left(\frac{b_{31}}{2} + \frac{b_{42}}{2}\right) + e_{4} \left(\frac{b_{41}}{2} + \frac{b_{52}}{2}\right), \  \frac{b_{13} e_{0}}{3} + \frac{b_{51} e_{6}}{3} + e_{1} \left(\frac{b_{12}}{3} + \frac{b_{23}}{3}\right) + e_{2} \left(\frac{b_{11}}{3} + \frac{b_{22}}{3} + \frac{b_{33}}{3}\right) + e_{3} \left(\frac{b_{21}}{3} + \frac{b_{32}}{3} + \frac{b_{43}}{3}\right) + e_{4} \left(\frac{b_{31}}{3} + \frac{b_{42}}{3} + \frac{b_{53}}{3}\right) + e_{5} \left(\frac{b_{41}}{3} + \frac{b_{52}}{3}\right), \  \frac{b_{14} e_{0}}{4} + \frac{b_{51} e_{7}}{4} + e_{1} \left(\frac{b_{13}}{4} + \frac{b_{24}}{4}\right) + e_{2} \left(\frac{b_{12}}{4} + \frac{b_{23}}{4} + \frac{b_{34}}{4}\right) + e_{3} \left(\frac{b_{11}}{4} + \frac{b_{22}}{4} + \frac{b_{

In [18]:
gab_k = diagonal_averaging(AEB)
print_latex(gab_k)

\left[ a_{11} b_{11} e_{0} + a_{15} b_{51} e_{8} + e_{1} \left(a_{11} b_{21} + a_{12} b_{11}\right) + e_{2} \left(a_{11} b_{31} + a_{12} b_{21} + a_{13} b_{11}\right) + e_{3} \left(a_{11} b_{41} + a_{12} b_{31} + a_{13} b_{21} + a_{14} b_{11}\right) + e_{4} \left(a_{11} b_{51} + a_{12} b_{41} + a_{13} b_{31} + a_{14} b_{21} + a_{15} b_{11}\right) + e_{5} \left(a_{12} b_{51} + a_{13} b_{41} + a_{14} b_{31} + a_{15} b_{21}\right) + e_{6} \left(a_{13} b_{51} + a_{14} b_{41} + a_{15} b_{31}\right) + e_{7} \left(a_{14} b_{51} + a_{15} b_{41}\right), \  e_{0} \left(\frac{a_{11} b_{12}}{2} + \frac{a_{21} b_{11}}{2}\right) + e_{1} \left(\frac{a_{11} b_{22}}{2} + \frac{a_{12} b_{12}}{2} + \frac{a_{21} b_{21}}{2} + \frac{a_{22} b_{11}}{2}\right) + e_{2} \left(\frac{a_{11} b_{32}}{2} + \frac{a_{12} b_{22}}{2} + \frac{a_{13} b_{12}}{2} + \frac{a_{21} b_{31}}{2} + \frac{a_{22} b_{21}}{2} + \frac{a_{23} b_{11}}{2}\right) + e_{3} \left(\frac{a_{11} b_{42}}{2} + \frac{a_{12} b_{32}}{2} + \frac{a_{13} 